In [2]:


from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
import pickle
import os.path
import re 
from collections import defaultdict
from collections import Counter
import datetime
import codecs
from os import listdir
from os.path import isfile, join

def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [33]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request is None:
            print ('Found None object')
            break
        if request.status_code == 401:
            print ('Unauthorized data access. The user does not have access to such information')
            break
#         elif "page does not exist" in request.text:
#             print 'page does not exist'
#             break
        elif request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 1 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 1)

In [34]:
def get_followers():
    followers = []
#     f = open ('iit_followers.txt','r')
    f = open ('nu_followers.txt','r')
    regexp = re.compile(r'p\d+')
    for line in f:
        #print(str(line))
        if regexp.search(str(line)) is not None:
            continue
        else: 
            if (str(line)).startswith('aV'):
                line = line[2:]
                followers.append(line.replace("\n",""))
    followers.sort()
    return followers

followers_list = get_followers()
print (len(followers_list))

35885


In [35]:
#get first 100 user IDs
userIds = []
startIndex = 0
endIndex = 100
while endIndex < len(followers_list):
    userInfo = twitter.request('users/lookup', {'screen_name': followers_list[startIndex:endIndex]})
    for user in userInfo:
        userIds.append(user['id'])
    if(len(followers_list) > 10000):
        #skip over 300 tweets (get 1/4 only)
        startIndex += 400
        endIndex += 400
    else:
        startIndex += 100
        endIndex += 100
    print ('processed', startIndex, ' users...')
print ('done')
print (len(userIds))

processed 400  users...
processed 800  users...
processed 1200  users...
processed 1600  users...
processed 2000  users...
processed 2400  users...
processed 2800  users...
processed 3200  users...
processed 3600  users...
processed 4000  users...
processed 4400  users...
processed 4800  users...
processed 5200  users...
processed 5600  users...
processed 6000  users...
processed 6400  users...
processed 6800  users...
processed 7200  users...
processed 7600  users...
processed 8000  users...
processed 8400  users...
processed 8800  users...
processed 9200  users...
processed 9600  users...
processed 10000  users...
processed 10400  users...
processed 10800  users...
processed 11200  users...
processed 11600  users...
processed 12000  users...
processed 12400  users...
processed 12800  users...
processed 13200  users...
processed 13600  users...
processed 14000  users...
processed 14400  users...
processed 14800  users...
processed 15200  users...
processed 15600  users...
processed 16

In [35]:
userInfo = twitter.request('users/lookup', {'screen_name': 'hpandya2_pa'})
for user in userInfo:
    userIds.insert(0, user['id'])
    print (user['id'])

2885816063


In [36]:
def get_users():
    print ('in get_users()')
    request = twitter.request('statuses/filter', {'follow':userIds[:5000], 'filter_level':'low'})
    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

    newfile = 'streaming_user_tweets_nu_' + timestamp + '.bin'
    numTweets = 0
    print ('starting collection!!! :)')
    
    while True:
        try:
            for tweet in request:
                print ("got a tweet")
                try:
                    print (tweet['text'])
                    print ('')
                except:
                    continue
                pickle.dump(tweet, open(newfile, 'ab'))
                numTweets += 1
                print ('dumped tweet to file')
                if numTweets > 5000:
                    numTweets = 1
                    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
                    newfile = 'streaming_user_tweets_' + timestamp + '.bin' 
        except:
            print ('Got error: (sleeping for 60 seconds...)')
            print (request.text)
            sys.stderr.flush()
            time.sleep(60) 
               
get_users()               

in get_users()
starting collection!!! :)
got a tweet
RT @GE_Foundation: How #apprenticeships can help solve the #skillsgap: https://t.co/dCoSGS42Dz

dumped tweet to file
got a tweet
RT @Ostrov_A: Wishing all #ShabbatShalom, especially in #Israel, where we pray it is a peaceful #Shabbat.

dumped tweet to file
got a tweet
Find out more about @IL_Hunger and they work the do in #Chicago in the #StreetWise #Nonprofit guide! https://t.co/aMIDdSDZb9

dumped tweet to file
got a tweet
RT @ColeyHarvey: Think the Bengals are tight? Before Friday's walkthrough, righty Andy Dalton throws... https://t.co/kq769zzpUy https://t.c…

dumped tweet to file
got a tweet
RT @Ostrov_A: #BDSFail of week! Pro-#BDS hate group 'Students for Justice in #Palestine' uses #Israeli co. @Wix to build website https://t.…

dumped tweet to file
got a tweet
https://t.co/Wwx5SV1JfY

#EURUSD  &lt; 1.06 pour finir la semaine
en attendant #BCE &amp; #FED en Decembre https://t.co/XVdk5DpwQi

dumped tweet to file
got a tweet
RT @

KeyboardInterrupt: 

In [23]:
def readFileAndLabelTweets(skipFirstNTweets):
    tweetFiles = ['streaming_user_tweets_2015-11-21 01:20:40.bin',
                  'streaming_user_tweets_2015-11-23 00:18:26.bin']
    tweetNumber = 1
    tweets = []
    for file in tweetFiles:
        print ('getting tweets from file', file)
        openedFile = open(file, 'rb')
        while True:   
            try:    
                loadedTweet = pickle.load(openedFile)
                
                if skipFirstNTweets + 1 > tweetNumber:
                    tweetNumber+= 1
                    continue
                
                if 'text' in loadedTweet:
                    thisTweet = loadedTweet['text']
                    #only look at unique tweets during classification
                    if thisTweet in tweets:
                        continue
                    else:
                        tweets.append(thisTweet)

                    print ("%d) %s\n" % (tweetNumber, thisTweet))
                    #classify...
                    label = input('Tweet label (1=good, 2=bad, 3=irrelevant, 4=stop): ')
                    if label == '1':
                        print('good\n')
                    elif label == '2':
                        print('bad\n')
                    elif label == '3':
                        print('irrelevant\n')
                    elif label == '4':
                        print('stopping\n')
                        print('labeled %d tweets, ending with tweet #%d' % ((len(tweets) - 1),(tweetNumber - 1)))
                        return
                    
                    #save tweet text with a label
                    tweetTextWithLabel = {'label': label, 'tweetText': loadedTweet['text']}
                    fileName = 'labelsFor' + file
                    pickle.dump(tweetTextWithLabel, open(fileName, 'ab'))
                    
                    tweetNumber += 1
                else:
                    continue
            except EOFError:
                print ("end of file...exiting")
                break

#allow to resume adding labels from the middle of tweets
skipFirstNTweets = 2
readFileAndLabelTweets(skipFirstNTweets)


getting tweets from file streaming_user_tweets_2015-11-21 01:20:40.bin
3) RT @HarvardBiz: You're more likely to believe something if there's a chart. That's not always a good thing: https://t.co/KvSB9cA7QD https:/…

Tweet label (1=good, 2=bad, 3=irrelevant, 4=stop): 4
stopping

labeled 0 tweets, ending with tweet #2


In [39]:
def convertTweetsToPython2():
    #tweetFiles = ['streaming_user_tweets_2015-11-21 01:20:40.bin',
    #              'streaming_user_tweets_2015-11-23 00:18:26.bin']
    tweetFiles = ['streaming_user_tweets_nu_2015-11-27 10:35:31.bin']
    tweets = []
    for file in tweetFiles:
        print ('getting tweets from file', file)
        openedFile = open(file, 'rb')
        while True:   
            try:    
                loadedTweet = pickle.load(openedFile)
                
                if 'text' in loadedTweet:
                    thisTweet = loadedTweet['text']
                    #only look at unique tweets during classification
                    if thisTweet in tweets:
                        continue
                    else:
                        tweets.append(thisTweet)
                    
                    fileName = 'convertedNUTweets.bin'
                    pickle.dump(loadedTweet, open(fileName, 'ab'), protocol=2)
                
                else:
                    continue
            except EOFError:
                print ("end of file...exiting")
                break
    print('done - converted', len(tweets), 'unique tweets')

#allow to resume adding labels from the middle of tweets
convertTweetsToPython2()

getting tweets from file streaming_user_tweets_nu_2015-11-27 10:35:31.bin
end of file...exiting
done - converted 4171 unique tweets


In [19]:
def readTweetTextWithLabel(tweetFiles):
    #Just IIT
    #'labelsFor_convertedIITTweets.bin',
    #'labelsFor_convertedNUTweets.bin',
    #tweetFiles = ['labelsFor_convertedIITTweets.bin']
    #Just NU
    #tweetFiles = ['labelsFor_pickle_nu_outputfile_2015-11-26 12_58_06.txt', 'labelsFor_pickle_nu_outputfile_2015-11-26 14_28_24.txt']
    #Both IIT and NU
    #tweetFiles = ['labelsFor_convertedIITTweets.bin', 'labelsFor_pickle_nu_outputfile_2015-11-26 12:58:06.txt', 'labelsFor_pickle_nu_outputfile_2015-11-26 14:28:24.txt','labelsFor_pickle_outputfile_2015-11-17 18:32:38.txt','labelsFor_pickle_outputfile_2015-11-17 20:57:07.txt'] 
    tweetNumber = 1
    tweets = []
    tweetsWithLabels = []
    for file in tweetFiles:
        print ('getting tweets with label from file', file)
        openedFile = open(file, 'rb')
        while True:   
            try:    
                loadedTweet = pickle.load(openedFile)
                
                #this will only run if multiple labels given to same tweet
                if 'tweetText' in loadedTweet:
                    thisTweetText = loadedTweet['tweetText']
                    #only look at unique tweets during classification
                    if thisTweetText in tweets:
                        continue
                    else:
                        tweets.append(thisTweetText)
                        tweetsWithLabels.append(loadedTweet)

                    labelText = ''
                    if loadedTweet['label'] == '1':
                        labelText = '[academics]'
                    elif loadedTweet['label'] == '2':
                        labelText = '[sports]'
                    elif loadedTweet['label'] == '3':
                        labelText = '[politics]'
                    elif loadedTweet['label'] == '4':
                        labelText = '[technology]'
                    elif loadedTweet['label'] == '5':
                        labelText = '[current_affairs]'
                    elif loadedTweet['label'] == '6':
                        labelText = '[irrelevant]'
                    elif loadedTweet['label'] == '7':
                        labelText = '[campus]'    
                    
                    #print ("%d) %-10s %s\n" % (tweetNumber, labelText, thisTweetText))
                    tweetNumber += 1
                else:
                    continue
            except EOFError:
                print ("end of file...exiting")
                break
                
    return tweetsWithLabels
                
tweetsWithLabels = readTweetTextWithLabel(['labelsFor_convertedIITTweets.bin', 'labelsFor_pickle_nu_outputfile_2015-11-26 12:58:06.txt', 'labelsFor_pickle_nu_outputfile_2015-11-26 14:28:24.txt','labelsFor_pickle_outputfile_2015-11-17 18:32:38.txt','labelsFor_pickle_outputfile_2015-11-17 20:57:07.txt'])

('getting tweets with label from file', 'labelsFor_convertedIITTweets.bin')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_nu_outputfile_2015-11-26 12:58:06.txt')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_nu_outputfile_2015-11-26 14:28:24.txt')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_outputfile_2015-11-17 18:32:38.txt')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_outputfile_2015-11-17 20:57:07.txt')
end of file...exiting


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import numpy
from collections import defaultdict

def accuracy(trueLabel, predictedLabel):
    return (1. * len([True for t, p in zip(trueLabel, predictedLabel) if t == p]) / len(trueLabel))

def textLabel(num):
    labelText = ''
    if num == 1:
        labelText = 'academics (1)'
    elif num == 2:
        labelText = 'sports (2)'
    elif num == 3:
        labelText = 'politics (3)'
    elif num == 4:
        labelText = 'technology (4)'
    elif num == 5:
        labelText = 'current_affairs (5)'
    elif num == 6:
        labelText = 'irrelevant (6)'
    elif num == 7:
        labelText = 'campus (7)'
    return labelText

def printIncorrectlyLabeledTweets(index, trueLabel, predictedLabel, tweetText):
    if trueLabel != predictedLabel:
        print('')
        print(index)
        print ('--- ACTUAL:', textLabel(trueLabel), 'PREDICTED:', textLabel(predictedLabel), '---')
        print (tweetText)
        return True
    return False

def applyOverrides(labeledTweets):
    labeledTweets[2]['label'] = '4'
    labeledTweets[4]['label'] = '6'
    labeledTweets[13]['label'] = '6'  
    labeledTweets[14]['label'] = '6'  
    labeledTweets[18]['label'] = '6'
    labeledTweets[20]['label'] = '6'
    labeledTweets[22]['label'] = '6'
    labeledTweets[26]['label'] = '6'
    labeledTweets[37]['label'] = '6'
    labeledTweets[38]['label'] = '6'
    labeledTweets[39]['label'] = '6'
    labeledTweets[67]['label'] = '6'
    labeledTweets[76]['label'] = '6'
    labeledTweets[136]['label'] = '6'
    labeledTweets[141]['label'] = '6'
    return labeledTweets
    
def createXandYVectors(labeledTweets):
    vectorizer = CountVectorizer(min_df=2)
    X = vectorizer.fit_transform(tweet['tweetText'] for tweet in labeledTweets)
    y = numpy.array([int(tweet['label']) for tweet in labeledTweets])
    return X, y

tweetsWithLabels = readTweetTextWithLabel(['labelsFor_convertedIITTweets.bin', 'labelsFor_pickle_nu_outputfile_2015-11-26 12:58:06.txt', 'labelsFor_pickle_nu_outputfile_2015-11-26 14:28:24.txt','labelsFor_pickle_outputfile_2015-11-17 18:32:38.txt','labelsFor_pickle_outputfile_2015-11-17 20:57:07.txt'])
X, y = createXandYVectors(tweetsWithLabels)

# print(y[0], y[1], y[2], y[3], y[4], y[5])
# print(tweetsWithLabels[0]['label'], tweetsWithLabels[1]['label'], tweetsWithLabels[2]['label'], tweetsWithLabels[3]['label'], tweetsWithLabels[4]['label'], tweetsWithLabels[5]['label'])

#model across all labeled data
model = LogisticRegression()
X, y = createXandYVectors(tweetsWithLabels)
model.fit(X, y)
predictions = model.predict(X)
print('full tweet count:', len(y))
print('accuracy fitting to all data:', accuracy(y, predictions))

#inspect accuracy of model using KFold
accuracyPerFold = []
for trainingIndex, testingIndex in KFold(len(y), 5):
    model.fit(X[trainingIndex], y[trainingIndex])
    predictions = model.predict(X[testingIndex])
    accuracyPerFold.append(accuracy(y[testingIndex], predictions))

print ('accuracy per 5-fold:', accuracyPerFold)
print ('average accuracy per fold:', numpy.mean(accuracyPerFold))

print('')
#Model again, filtering out all irrelivant tweets
tweetsWithLabels2 = readTweetTextWithLabel(['labelsFor_convertedIITTweets.bin', 'labelsFor_pickle_nu_outputfile_2015-11-26 12:58:06.txt', 'labelsFor_pickle_nu_outputfile_2015-11-26 14:28:24.txt','labelsFor_pickle_outputfile_2015-11-17 18:32:38.txt','labelsFor_pickle_outputfile_2015-11-17 20:57:07.txt'])

filteredTweets = [tweet for tweet in tweetsWithLabels2 if tweet['label'] != '6']
X, y = createXandYVectors(filteredTweets)
model.fit(X, y)
predictions = model.predict(X)
print('reduced tweet count:', len(y))
print('accuracy fitting to all data:', accuracy(y, predictions))

#inspect accuracy of model using KFold

accuracyPerFold = []
for trainingIndex, testingIndex in KFold(len(y), 5):
    model.fit(X[trainingIndex], y[trainingIndex])
    predictions = model.predict(X[testingIndex])
    accuracyPerFold.append(accuracy(y[testingIndex], predictions))
    for i, index in enumerate(testingIndex):
        r = printIncorrectlyLabeledTweets(index, y[index], predictions[i], filteredTweets[index]['tweetText'])
        
print('')
print ('accuracy per 5-fold:', accuracyPerFold)
print ('average accuracy per fold:', numpy.mean(accuracyPerFold))

# X, y = createXandYVectors(filteredTweets)
# model.fit(X, y)
# predictions = model.predict(X)
#     X = vectorizer.fit_transform(tweet['text'] for tweet in tweets)


('getting tweets with label from file', 'labelsFor_convertedIITTweets.bin')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_nu_outputfile_2015-11-26 12:58:06.txt')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_nu_outputfile_2015-11-26 14:28:24.txt')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_outputfile_2015-11-17 18:32:38.txt')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_outputfile_2015-11-17 20:57:07.txt')
end of file...exiting
('full tweet count:', 1969)
('accuracy fitting to all data:', 0.9507364144235653)
('accuracy per 5-fold:', [0.7893401015228426, 0.8299492385786802, 0.7791878172588832, 0.8274111675126904, 0.8549618320610687])
('average accuracy per fold:', 0.81617003138683297)

('getting tweets with label from file', 'labelsFor_convertedIITTweets.bin')
end of file...exiting
('getting tweets with label from file', 'labelsFor_pickle_nu_outputf

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import numpy
from collections import defaultdict

def ReadPickledTweets(filenames):
    tweets = []
    for filename in filenames:
        f = open(filename,'rb')
        while True:
            try:
                loadedtweet = pickle.load(f)
                tweets.append(loadedtweet)
            except EOFError:
                print 'end of file... exiting'
                break
    return tweets
#,max_df=.7,stop_words={'english'},lowercase=True,,max_features=680
def create_school_matrix(list_of_files):
    list_of_tweets = ReadPickledTweets(list_of_files)
    vectorizer = CountVectorizer(min_df=2,max_features=680)
    X = vectorizer.fit_transform(tweet['text'] for tweet in list_of_tweets)
    return X

# IIT_X = create_school_matrix(['pickle_outputfile_2015-11-17 22:02:04.txt',
#                              'pickle_outputfile_2015-11-17 23:47:46.txt',
#                              'pickle_outputfile_2015-11-18 00:54:18.txt',
#                              'pickle_outputfile_2015-11-18 02:16:14.txt',
#                              'pickle_outputfile_2015-11-18 03:22:51.txt',
#                              'pickle_outputfile_2015-11-18 04:29:30.txt',
#                              'pickle_outputfile_2015-11-18 05:52:00.txt',
#                              'pickle_outputfile_2015-11-18 06:41:09.txt',
#                              'pickle_outputfile_2015-11-18 08:40:01.txt',
#                              'pickle_outputfile_2015-11-26 12:53:45.txt'])
# IIT_X.shape
# NU_X = create_school_matrix(['pickle_nu_outputfile_2015-11-26 15:18:08.txt',
#                             'pickle_nu_outputfile_2015-11-26 16:23:09.txt',
#                             'pickle_nu_outputfile_2015-11-26 17:28:03.txt',
#                             'pickle_nu_outputfile_2015-11-26 18:32:47.txt'])
    

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import numpy
IIT_X = create_school_matrix(['pickle_outputfile_2015-11-17 22:02:04.txt',
                             'pickle_outputfile_2015-11-17 23:47:46.txt',
                             'pickle_outputfile_2015-11-18 00:54:18.txt',
                             'pickle_outputfile_2015-11-18 02:16:14.txt'])
#                              'pickle_outputfile_2015-11-18 03:22:51.txt',
#                              'pickle_outputfile_2015-11-18 04:29:30.txt',
# IIT_X = create_school_matrix(['pickle_outputfile_2015-11-18 05:52:00.txt',
#                              'pickle_outputfile_2015-11-18 06:41:09.txt',
#                              'pickle_outputfile_2015-11-18 08:40:01.txt',
#                              'pickle_outputfile_2015-11-26 12:53:45.txt'])

print IIT_X.shape


end of file... exiting
end of file... exiting
end of file... exiting
end of file... exiting
(40039, 680)


In [52]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
import numpy
NU_X = create_school_matrix(['pickle_nu_outputfile_2015-11-26 15:18:08.txt',
                            'pickle_nu_outputfile_2015-11-26 16:23:09.txt',
                            'pickle_nu_outputfile_2015-11-26 17:28:03.txt',
                            'pickle_nu_outputfile_2015-11-26 18:32:47.txt'])

print NU_X.shape

end of file... exiting
end of file... exiting
end of file... exiting
end of file... exiting
(40053, 680)


In [83]:
#first four
# printing IIT_predictions
# sum1 40039
# 1 = 0.136416993431
# 2 = 0.296960463548
# 3 = 0.308024675941
# 4 = 0.108793925922
# 5 = 0.0662104448163
# 7 = 0.0835934963411
# printing NU_predictions
# sum2 40053
# 1 = 0.125608568647
# 2 = 0.431628092777
# 3 = 0.186652685192
# 4 = 0.0599455721169
# 5 = 0.113524579932
# 7 = 0.0826405013357

#all iit and four NU
# (372, 680)
# printing IIT_predictions
# sum1 72352
# 1 = 0.0744277974348
# 2 = 0.330370964175
# 3 = 0.0670333923043
# 4 = 0.109948584697
# 5 = 0.25145123839
# 7 = 0.166768022999
# printing NU_predictions
# sum2 40053
# 1 = 0.125608568647
# 2 = 0.431628092777
# 3 = 0.186652685192
# 4 = 0.0599455721169
# 5 = 0.113524579932
# 7 = 0.0826405013357

#last four IIT and first four NU
# (372, 680)
# printing IIT_predictions
# sum1 12285
# 1 = 0.0829466829467
# 2 = 0.375172975173
# 3 = 0.279365079365
# 4 = 0.0153846153846
# 5 = 0.0771672771673
# 7 = 0.169963369963
# printing NU_predictions
# sum2 40053
# 1 = 0.125608568647
# 2 = 0.431628092777
# 3 = 0.186652685192
# 4 = 0.0599455721169
# 5 = 0.113524579932
# 7 = 0.0826405013357
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
X, y = createXandYVectors(filteredTweets)
print X.shape
model.fit(X, y)

IIT_predictions = model.predict(IIT_X)
print 'printing IIT_predictions'
c1 = Counter(IIT_predictions)
sum1 = sum(c1.values())


NU_predictions = model.predict(NU_X)
print 'printing NU_predictions'
c2 = Counter(NU_predictions)
sum2 = sum(c2.values())

print 'IIT'
perc1 = []
for c in c1:
    perc1.append(float((c1[c])/sum1)*100)
    print '{0} = {1}'.format(c,(float(c1[c])/sum1)*100)

print '\n'
print 'NU'
perc2 = []
for c in c2:
    perc2.append(float((c2[c])/sum1)*100)
    print '{0} = {1}'.format(c,(float(c2[c])/sum2)*100)

print 'perc1' , sum(perc1)
print 'perc2' , sum(perc2)

labels = ['Academics', 'Sports', 'Politics', 'Technology', 'Current_Events', 'Campus_Events']

# ind = np.arange(6)  # the x locations for the groups
# width = 0.35       
# fig, ax = plt.subplots()
# rects1 = ax.bar(ind, labels, width, color='b', yerr= perc1)
# rects2 = ax.bar(ind +width, labels, width, color='y', yerr=perc2)
# ax.set_ylabel('percentages')
# ax.set_xticks(ind + width)
# ax.set_xticklabels(labels)

# def autolabel(rects):
#     # attach some text labels
#     for rect in rects:
#         height = rect.get_height()
#         ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
#                 '%d' % int(height),
#                 ha='center', va='bottom')

# autolabel(rects1)
# autolabel(rects2)

# plt.plot(labels, perc1)
# plt.show()


# plt.show()

(372, 680)
printing IIT_predictions
printing NU_predictions
IIT
1 = 8.29466829467
2 = 37.5172975173
3 = 27.9365079365
4 = 1.53846153846
5 = 7.71672771673
7 = 16.9963369963


NU
1 = 12.5608568647
2 = 43.1628092777
3 = 18.6652685192
4 = 5.99455721169
5 = 11.3524579932
7 = 8.26405013357
perc1 0.0
perc2 100.0


In [ ]:
 Category
                IIT (% tweets)     Northwestern(% tweets)
Academics
                8.29                12.56
Sports          37.51               43.162
Politics        27.93               18.66
Technology       1.53                 5.99
Current_Events   7.71                11.35
Campus_Events   16.99                 8.264
